In [6]:
import docker
import uuid
import paramiko
import os
from timeit import default_timer as timer
from dataclasses import dataclass
import re
import requests
import statistics
import pandas as pd
import numpy as np

In [7]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def run_in_hive(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("hive-server", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"bash -c '. /env_var_path.sh && {command}'")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

In [8]:
def setup_test_table():
    return run_in_hive("hive -f /data/master_volume/hive_scripts/employee_table.hql")
    
def copy_test_file():
    path = "/data/master_volume/hive_scripts/employee.csv"
    dest = "/user/hive/warehouse/testdb.db/employee"
    run_in_master(f"hdfs dfs -mkdir -p {dest}")
    return run_in_master(f"hdfs dfs -put {path} {dest}")

In [11]:
setup_test_table()
copy_test_file()

([],
 ["put: `/user/hive/warehouse/testdb.db/employee/employee.csv': File exists\n"])

In [12]:
run_in_hive("hive -f /data/master_volume/hive_scripts/test_group.hql")

([],
 ['SLF4J: Class path contains multiple SLF4J bindings.\n',
  'SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.10.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: Found binding in [jar:file:/opt/hadoop-3.3.1/share/hadoop/common/lib/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]\n',
  'SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.\n',
  'SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]\n',
  'Hive Session ID = 4e7c2ff0-33e9-4acf-996e-a99711bcbfbe\n',
  '\n',
  'Logging initialized using configuration in file:/opt/hive/conf/hive-log4j2.properties Async: true\n',
  'Hive Session ID = 4ebe5a73-cb05-424f-9d37-f3b524a58b07\n',
  'OK\n',
  'Time taken: 0.391 seconds\n',
  'Query ID = root_20230525185710_c1385f1a-8b71-463a-99fa-d06ad3204705\n',
  'Total jobs = 1\n',
  'Launching Job 1 out of 1\n',
  'Number of reduce tasks not specified. Estimated from input data size: 3\n',